In [2]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
#import xgboost as xgb
import lightgbm as lgb

In [3]:
data1 = pd.read_csv('../data/data_label1.csv')

In [7]:
vec = data1[['aid','interest1', 'interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']]
vec.head()

,aid,interest1,interest2,interest5,kw1,kw2,topic1,topic2
0,1119,93 70 77 109 121 47 75 1 102 29 49 6 42 73 46 ...,24 6 19 13 30,100 80 37 116 71 73 6 46 62 59 118 58 129 114 65,338851 463925 691885 80873 350047,74061 67582 44005 14660 104708,6939 8366 1338 1450 5955,3274 3865 4159 7924 8465
1,1335,93 70 77 76 48 46 36 11,-1,-1,727009 338851 68812 8561 391535,87384 81211 97866 85039 56336,7318 257 5828 8048 8402,1052 1825 8112 1188 3755
2,1119,93 70 86 37 75 4 69 29 83 6 42 104 46 36 11 96...,24 30,-1,172567 10251 634625 388504 527776,32481 6918 44999 28033 47192,2123 5828 4771 3014 4388,6269 3632 1570 3540 7555
3,1415,70 30 77 75 76 8 29 67 27 42 92 46 35 17 56 36...,47 24 73 32 33 19 13 30 74 45,127 52 72 80 31 92 131 37 116 47 12 8 69 28 6 ...,688291 67406 122910 75722 18022,51399 104182 64220 45950 49193,5338 7608 5935 7604 7456,7366 1094 2902 5188 1892
4,369,70 109 76 16 59 49 122 6 73 111 60 113 57 116 ...,24 46 30 72,52 131 37 13 71 8 50 6 76 27 46 126 59 60 56 1...,91104 614872 420403 778320 612659,95439 3372 117660 79604 18185,5840 3814 5997 4940 117,8917 41 4793 3298 1063


In [32]:
vec_g = vec.groupby('aid')
vec_g.head()

,aid,interest1,interest2,interest5,kw1,kw2,topic1,topic2
0,1119,93 70 77 109 121 47 75 1 102 29 49 6 42 73 46 ...,24 6 19 13 30,100 80 37 116 71 73 6 46 62 59 118 58 129 114 65,338851 463925 691885 80873 350047,74061 67582 44005 14660 104708,6939 8366 1338 1450 5955,3274 3865 4159 7924 8465
1,1335,93 70 77 76 48 46 36 11,-1,-1,727009 338851 68812 8561 391535,87384 81211 97866 85039 56336,7318 257 5828 8048 8402,1052 1825 8112 1188 3755
2,1119,93 70 86 37 75 4 69 29 83 6 42 104 46 36 11 96...,24 30,-1,172567 10251 634625 388504 527776,32481 6918 44999 28033 47192,2123 5828 4771 3014 4388,6269 3632 1570 3540 7555
3,1415,70 30 77 75 76 8 29 67 27 42 92 46 35 17 56 36...,47 24 73 32 33 19 13 30 74 45,127 52 72 80 31 92 131 37 116 47 12 8 69 28 6 ...,688291 67406 122910 75722 18022,51399 104182 64220 45950 49193,5338 7608 5935 7604 7456,7366 1094 2902 5188 1892
4,369,70 109 76 16 59 49 122 6 73 111 60 113 57 116 ...,24 46 30 72,52 131 37 13 71 8 50 6 76 27 46 126 59 60 56 1...,91104 614872 420403 778320 612659,95439 3372 117660 79604 18185,5840 3814 5997 4940 117,8917 41 4793 3298 1063
5,894,93 70 77 37 109 98 75 33 28 29 42 46 111 35 36 11,58 79 24 73 1 46 70 6 19 28 13 30 74 43 29,127 52 100 80 131 37 78 8 30 6 46 74 59 129,338851 497092 224284 410690 100680,104708 105941 106673 60043 51247,6939 2036 6816 9826 1379,1944 3279 6116 7403 4295
6,1468,93 46 36 11,-1,100 6 46 129,26682,46123,8176 1525 4197 1646 7704,525 6374 1188 9163 9775
7,2031,73 36 11 87,-1,72 33 131 11 109 8 66 6 76 89 59 129 93,636275 675598 530883 210500 85622,37879 51886 79676 92902 4721,1015 5302 7661 9826 8402,8048 1195 3726 926 7388
8,420,70 86 76 75 4 28 106 29 59 49 41 27 64 6 2 92 ...,47 24 46 32 30 37 29,52 72 131 11 4 48 8 113 28 66 6 42 76 20 46 59...,330721 627369 259755 314149 529100,28206 102474 88214 78971 36903,4812 4801 2583 7375 9249,2970 509 7509 5389 3706
9,1379,70 109 75 76 28 29 59 49 41 64 6 119 114 17 56...,22 9 70,72 13 79 6 42 27 9 89 74 59 84 64 129 35 40,666079 394919 182185 89154 272264,79250 77181 117548 18185 27802,5915 3207 607 7800 6096,585 1732 7612 1014 9968


In [33]:
vec['interest1'].value_counts()[:40]

-1                   43255
36 11                13970
70 76 36 11           3204
75 29 36 11           3195
70 76                 3102
75 29                 2142
93 46 36 11           1527
109 119               1394
59 49 36 11           1062
47 9                  1046
59 49                  915
17 56 36 11            906
109 119 36 11          892
93 46                  889
47 9 36 11             813
70 28 36 11            786
93 75 29 46 36 11      784
70 28                  651
93 70 76 46 36 11      624
70 76 28 36 11         615
17 56                  576
109 39                 480
6 116 36 11            465
109 39 36 11           452
49 18                  437
6 116                  434
86 2                   394
70 76 75 29 36 11      388
77 46                  374
49 114                 370
77 46 36 11            368
70 76 59 49 36 11      339
118 8 36 11            339
86 83                  336
49 23                  335
118 8                  330
86 2 36 11             329
7

In [34]:
all_interest1 = vec['interest1'].value_counts().index
all_interest1 = all_interest1.tolist()
interests1 = dict()
for i in np.array(all_interest1):
    data = i.split(' ')
    for j in data:
        interests1[j] = interests1.get(j, 0) + 1
sort1 = sorted(interests1.items(),key = lambda x:x[1],reverse = True)

In [35]:
all_interest1

['-1',
 '36 11',
 '70 76 36 11',
 '75 29 36 11',
 '70 76',
 '75 29',
 '93 46 36 11',
 '109 119',
 '59 49 36 11',
 '47 9',
 '59 49',
 '17 56 36 11',
 '109 119 36 11',
 '93 46',
 '47 9 36 11',
 '70 28 36 11',
 '93 75 29 46 36 11',
 '70 28',
 '93 70 76 46 36 11',
 '70 76 28 36 11',
 '17 56',
 '109 39',
 '6 116 36 11',
 '109 39 36 11',
 '49 18',
 '6 116',
 '86 2',
 '70 76 75 29 36 11',
 '77 46',
 '49 114',
 '77 46 36 11',
 '70 76 59 49 36 11',
 '118 8 36 11',
 '86 83',
 '49 23',
 '118 8',
 '86 2 36 11',
 '70 47 76 9 36 11',
 '49 18 36 11',
 '86 83 36 11',
 '49 114 36 11',
 '70 100 76 36 11',
 '70 76 17 56 36 11',
 '70 100',
 '70 76 106 36 11',
 '109 75 29 119 36 11',
 '73 36 11 87',
 '69 54',
 '93 77 46 36 11',
 '42 62 36 11',
 '70 75 76 29 36 11',
 '93 70 76 75 29 46 36 11',
 '59 49 17 56 36 11',
 '70 106 36 11',
 '70 76 28',
 '47 75 29 9 36 11',
 '42 62',
 '70 106',
 '75 29 59 49 36 11',
 '86 75 29 2 36 11',
 '49 15',
 '86 75 29 83 36 11',
 '6 57 36 11',
 '70 100 36 11',
 '70 109 76 119 

In [36]:
sort2

[('70', 80676),
 ('46', 63258),
 ('73', 60460),
 ('4', 46850),
 ('30', 45230),
 ('9', 40119),
 ('10', 40087),
 ('24', 39274),
 ('21', 39172),
 ('29', 37165),
 ('1', 36616),
 ('6', 36066),
 ('22', 35565),
 ('74', 32367),
 ('79', 31064),
 ('54', 31030),
 ('48', 23310),
 ('14', 22811),
 ('35', 22112),
 ('19', 21717),
 ('47', 21677),
 ('28', 21660),
 ('52', 21217),
 ('41', 19552),
 ('13', 19351),
 ('37', 18682),
 ('33', 18051),
 ('42', 17812),
 ('72', 17640),
 ('31', 15715),
 ('44', 14199),
 ('32', 13642),
 ('43', 13603),
 ('58', 13442),
 ('63', 12405),
 ('39', 10943),
 ('18', 10896),
 ('66', 9099),
 ('55', 6366),
 ('49', 5810),
 ('23', 5639),
 ('3', 5349),
 ('27', 5210),
 ('69', 5205),
 ('80', 5110),
 ('65', 4630),
 ('20', 4210),
 ('51', 3758),
 ('16', 3685),
 ('45', 3053),
 ('36', 2692),
 ('50', 2096),
 ('15', 2070),
 ('71', 1990),
 ('53', 1919),
 ('7', 1917),
 ('59', 1654),
 ('67', 1496),
 ('68', 1435),
 ('76', 1372),
 ('26', 1308),
 ('17', 1210),
 ('11', 1004),
 ('82', 981),
 ('40', 93

In [ ]:

num_vec = []
train_array = np.array(vec)

for i in range(train_array.shape[0]):
    num = 0
    inter = train_array[i]
    for j in inter:
        inter_lis = j.split(' ')
        if inter_lis[0] == '-1':
            continue
        num += len(inter_lis)
    num_vec.append(num)
    
num_all_vec  = pd.DataFrame(num_vec, columns=['num_all_vec'])

data =  pd.concat([data, num_all_vec], axis=1)
print('加入用户兴趣总数,OK!')